In [5]:
import requests
import cssutils
from bs4 import BeautifulSoup
import os
import json
import pandas as pd

link = 'https://store.steampowered.com/search/results'
game = input('Enter game you want search: ')
head = {'cookie': 'sessionid=5ae342dee8ccf71fc64e6471'}

In [6]:
def get_pagination(game):
    param = {
        'term': game,
        'page': 1,
    }
    total_item = 0
    req = requests.get(link, headers=head, params=param)
    soup = BeautifulSoup(req.text, 'html.parser')
    page_item = soup.find('div', 'search_pagination_right').find_all('a')

    try:
        total_item = int(page_item[4].text)
    except Exception:
        pass
        try:
            total_item = int(page_item[3].text)
        except Exception:
            pass
            try:
                total_item = int(page_item[2].text)
            except Exception:
                pass
                try:
                    total_item = int(page_item[1].text)
                except Exception:
                    pass
                    try:
                        total_item = int(page_item[0].text)
                    except Exception:
                        pass
    return 1 + total_item

# Game Scraping

In [7]:
def scrap_game_detail(game):
    count = 0
    datas = []

    for j in range(1, get_pagination(game)):
        param = {
            'term': game,
            'page': j,
        }
        req = requests.get(link, params=param, headers=head)
        soup = BeautifulSoup(req.text, 'html.parser')

        conten = soup.find('div', {'id': 'search_resultsRows'}).find_all('a')#tablodaki her bir oyun için
        for i in conten:
            #search page
            #url
            url = i['href']
            
            #title
            title = i.find('div', 'col search_name ellipsis').text.strip().replace('\n', ' ')

            #price
            try:
                price = i.find('div', 'col search_price responsive_secondrow').text.strip()
            except Exception:
                price = 'discount from ' + i.find('span', {'style': 'color: #888888;'}).text.replace(' ', '.') + ' to ' + i.find('div', 'col search_price discounted responsive_secondrow').find('br').next_sibling.strip() + f" ({i.find('div', 'col search_discount responsive_secondrow').text.replace('-', '').strip()})"
            if price == '':
                price = 'none'

            #release
            release = i.find('div', 'col search_released responsive_secondrow').text
            if release == '':
                release = 'none'

            #game page
            req_piece = requests.get(url, headers=head)
            soup_piece = BeautifulSoup(req_piece.text, 'html.parser')

            #description
            description = soup_piece.find("div", {"class" : "game_description_snippet"})
            if description == None:
                description = None 
            else:
                description = description.text.strip().replace('\n', ' ')

            #header image
            header_image = soup_piece.find("img", {"class" : "game_header_image_full"})
            if header_image == None:
                header_image = None
            else:
                header_image = header_image['src']
            
            #developer
            try:
                developer = soup_piece.find_all("div", {"class" : "dev_row"})[0]
                if developer == None:
                    developer = None
                else:
                    developer = developer.find("a").text
            except:
                developer = None
            
            #publisher
            try:
                publisher = soup_piece.find_all("div", {"class" : "dev_row"})[1]
                if developer == None:
                    publisher = None
                else:
                    publisher = publisher.find("a").text
            except:
                publisher = None
            
            #tags
            tags = soup_piece.find("div", {"class" : "glance_tags popular_tags"})
            if tags == None:
                tags = None
            else:
                tags = [i.text.strip().replace('\n', ' ') for i in tags.find_all("a")][:5]
              
            #languages
            try:  
                languages = soup_piece.find_all("div", {"id" : "languageTable"})
                if languages == None:
                    languages = None
                else:                    
                    languages = [i.text.strip().replace('\n', ' ') for i in languages[0].find_all("td", {"class":"ellipsis"})]
            
            except:
                languages = None
            
            #background image
            background_image = soup_piece.find("img", {"class" : "game_page_background game"})
            if background_image == None:
                background_image = None
            else:
                background_image = cssutils.parseStyle(background_image["style"])['background-image']
                background_image = background_image.replace('url(', '').replace(')', '')
                
            sys_req = soup_piece.find("div", {"class" : "game_area_sys_req sysreq_content active"})
            if sys_req == None:
                sys_req = None
            else:
                sys_req = sys_req.text.strip().replace('\n', ' ')

            data = {
                'title': title,
                'description': description,
                'price': price,
                'release': release,
                'link': url,
                'header_image': header_image,
                'developer': developer,
                'publisher': publisher,
                'tags':tags,
                'languages':languages,
                'background_image': background_image,
                'sys_req':sys_req
            }
            datas.append(data)

            count += 1
    return datas

In [192]:
def game_loop(gameList):
    all_game = list()
    for game in gameList:
        game_detail = scrap_game_detail(game)
        all_game.append(game_detail)
        
    return all_game

In [169]:
df = pd.read_csv("imdb_video_game_rating.csv")

In [193]:
gameList = df['title'].tolist()[:5]

In [ ]:
game_loop(gameList)

# Comment Scraping 

In [8]:
def scrap_comment_detail(game):
    count = 0
    datas = []

    for j in range(1, get_pagination(game)):
        param = {
            'term': game,
            'page': j,
        }
        req = requests.get(link, params=param, headers=head)
        soup = BeautifulSoup(req.text, 'html.parser')

        conten = soup.find('div', {'id': 'search_resultsRows'}).find_all('a')
        for i in conten:
            #search page
            #url
            url = i['href']
            
            #title
            title = i.find('div', 'col search_name ellipsis').text.strip().replace('\n', ' ')

            #price
            try:
                price = i.find('div', 'col search_price responsive_secondrow').text.strip()
            except Exception:
                price = 'discount from ' + i.find('span', {'style': 'color: #888888;'}).text.replace(' ', '.') + ' to ' + i.find('div', 'col search_price discounted responsive_secondrow').find('br').next_sibling.strip() + f" ({i.find('div', 'col search_discount responsive_secondrow').text.replace('-', '').strip()})"
            if price == '':
                price = 'none'

            #release
            release = i.find('div', 'col search_released responsive_secondrow').text
            if release == '':
                release = 'none'

            #game page
            req_piece = requests.get(url, headers=head)
            soup_piece = BeautifulSoup(req_piece.text, 'html.parser')

            #description
            description = soup_piece.find("div", {"class" : "game_description_snippet"})
            if description == None:
                description = None 
            else:
                description = description.text.strip().replace('\n', ' ')

            #header image
            header_image = soup_piece.find("img", {"class" : "game_header_image_full"})
            if header_image == None:
                header_image = None
            else:
                header_image = header_image['src']
            
            #developer
            try:
                developer = soup_piece.find_all("div", {"class" : "dev_row"})[0]
                if developer == None:
                    developer = None
                else:
                    developer = developer.find("a").text
            except:
                developer = None
            
            #publisher
            try:
                publisher = soup_piece.find_all("div", {"class" : "dev_row"})[1]
                if developer == None:
                    publisher = None
                else:
                    publisher = publisher.find("a").text
            except:
                publisher = None
            
            #tags
            tags = soup_piece.find("div", {"class" : "glance_tags popular_tags"})
            if tags == None:
                tags = None
            else:
                tags = [i.text.strip().replace('\n', ' ') for i in tags.find_all("a")][:5]
              
            #languages
            try:  
                languages = soup_piece.find_all("div", {"id" : "languageTable"})
                if languages == None:
                    languages = None
                else:                    
                    languages = [i.text.strip().replace('\n', ' ') for i in languages[0].find_all("td", {"class":"ellipsis"})]
            
            except:
                languages = None
            
            #background image
            background_image = soup_piece.find("img", {"class" : "game_page_background game"})
            if background_image == None:
                background_image = None
            else:
                background_image = cssutils.parseStyle(background_image["style"])['background-image']
                background_image = background_image.replace('url(', '').replace(')', '')
                
            sys_req = soup_piece.find("div", {"class" : "game_area_sys_req sysreq_content active"})
            if sys_req == None:
                sys_req = None
            else:
                sys_req = sys_req.text.strip().replace('\n', ' ')

            data = {
                'title': title,
                'description': description,
                'price': price,
                'release': release,
                'link': url,
                'header_image': header_image,
                'developer': developer,
                'publisher': publisher,
                'tags':tags,
                'languages':languages,
                'background_image': background_image,
                'sys_req':sys_req
            }
            datas.append(data)

            count += 1
    return datas

In [9]:
link = 'https://store.steampowered.com/search/results'
game = input('Enter game you want search: ')
head = {'cookie': 'sessionid=5ae342dee8ccf71fc64e6471'}

In [11]:
req = requests.get(link, headers=head)
soup = BeautifulSoup(req.text, 'html.parser')
conten = soup.find('div', {'id': 'search_resultsRows'}).find_all('a')

In [12]:
for i in conten:
    url = i['href']
    title = i.find('div', 'col search_name ellipsis').text.strip().replace('\n', ' ')

In [24]:
req_piece = requests.get(url, headers=head)
soup_piece = BeautifulSoup(req_piece.text, 'html.parser')

In [25]:
soup_piece.find_all("div", {"id" : "Reviews_summary"})

<!DOCTYPE html>

<html class="responsive" lang="en">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<meta content="#171a21" name="theme-color"/>
<title>Save 50% on PAYDAY 2 on Steam</title>
<link href="/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/motiva_sans.css?v=Rc2hpzg2Ex3T&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/shared_global.css?v=oXbBsQ8iQ0D-&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/css/buttons.css?v=6PFqex5UPprb&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/css/v6/store.css?v=Tsc-wpkmhmCc&amp;l=english" rel="stylesheet" type="text/css"/>
<link href="https://store.akamai.steamstatic.com/public/shared/c